# Debugg /optimisation
Les informations qaund spark tourne sont disponnible ici
http://localhost:4040/jobs/


In [1]:
from pyspark import SparkContext


sc = SparkContext()

#####RAW code to debug//optimize

def filter_stop_words(word):
    from nltk.corpus import stopwords
    english_stop_words = stopwords.words("english")
    return word not in english_stop_words

def load_text(text_path):
    # Split text in words
    # Remove empty word artefacts
    # Remove stop words ('I', 'you', 'a', 'the', ...)
    vocabulary = sc.textFile(text_path)\
        .flatMap(lambda lines: lines.lower().split())\
        .flatMap(lambda word: word.split("."))\
        .flatMap(lambda word: word.split(","))\
        .flatMap(lambda word: word.split("!"))\
        .flatMap(lambda word: word.split("?"))\
        .flatMap(lambda word: word.split("'"))\
        .flatMap(lambda word: word.split("\""))\
        .filter(lambda word: word is not None and len(word) > 0)\
        .filter(filter_stop_words)
        
    # Count the total number of words in the text
    word_count = vocabulary.count()

    # Compute the frequency of each word: frequency = #appearances/#word_count
    word_freq = vocabulary.map(lambda word: (word, 1))\
        .reduceByKey(lambda count1, count2: count1 + count2)\
        .map(lambda wc: (wc[0], wc[1]/float(word_count)))\

    return word_freq


def run():
    iliad = load_text('..\data/iliad.mb.txt')
    odyssey = load_text('..\data/odyssey.mb.txt')
    
    # Join the two datasets and compute the difference in frequency
    # Note that we need to write (freq or 0) because some words do not appear
    # in one of the two books. Thus, some frequencies are equal to None after
    # the full outer join.
    join_words = iliad.fullOuterJoin(odyssey)\
        .map(lambda wf: (wf[0], (wf[1][1] or 0) - (wf[1][0] or 0)))

    
    # 10 words that get a boost in frequency in the sequel
    emerging_words = join_words.takeOrdered(10, lambda wfd: -wfd[1])
    # 10 words that get a decrease in frequency in the sequel
    disappearing_words = join_words.takeOrdered(10, lambda wfd: -wfd[1])
    
    # Print results
    for word, freq_diff in emerging_words:
        print( "%.2f" % (freq_diff*10000), word)
    for word, freq_diff in disappearing_words[::-1]:
        print ("%.2f" % (freq_diff*10000), word)
    
   


In [2]:
run()

92.52 ulysses
53.63 house
48.33 telemachus
43.06 suitors
36.68 tell
33.47 ship
33.35 one
31.94 home
26.73 said
25.97 got
25.97 got
26.73 said
31.94 home
33.35 one
33.47 ship
36.68 tell
43.06 suitors
48.33 telemachus
53.63 house
92.52 ulysses


## Optimisation Job 2


.filter(filter_stop_words) devient
        
.filter(filter_stop_words)\
.persist()
        
Persist permet de persister les donnees et ne pas refaire les actions lors du count

In [5]:

def filter_stop_wordsOpt(word):
    from nltk.corpus import stopwords
    english_stop_words = stopwords.words("english")
    return word not in english_stop_words

def load_textOpt(text_path):
    # Split text in words
    # Remove empty word artefacts
    # Remove stop words ('I', 'you', 'a', 'the', ...)
    vocabulary = sc.textFile(text_path)\
        .flatMap(lambda lines: lines.lower().split())\
        .flatMap(lambda word: word.split("."))\
        .flatMap(lambda word: word.split(","))\
        .flatMap(lambda word: word.split("!"))\
        .flatMap(lambda word: word.split("?"))\
        .flatMap(lambda word: word.split("'"))\
        .flatMap(lambda word: word.split("\""))\
        .filter(lambda word: word is not None and len(word) > 0)\
        .filter(filter_stop_words)\
        .persist()
    # Count the total number of words in the text
    word_count = vocabulary.count()

    # Compute the frequency of each word: frequency = #appearances/#word_count
    word_freq = vocabulary.map(lambda word: (word, 1))\
        .reduceByKey(lambda count1, count2: count1 + count2)\
        .map(lambda wc: (wc[0], wc[1]/float(word_count)))\

    return word_freq


def runOpt():
    iliad = load_textOpt('..\data/iliad.mb.txt')
    odyssey = load_textOpt('..\data/odyssey.mb.txt')
    
    # Join the two datasets and compute the difference in frequency
    # Note that we need to write (freq or 0) because some words do not appear
    # in one of the two books. Thus, some frequencies are equal to None after
    # the full outer join.
    join_words = iliad.fullOuterJoin(odyssey)\
        .map(lambda wf: (wf[0], (wf[1][1] or 0) - (wf[1][0] or 0)))

    
    # 10 words that get a boost in frequency in the sequel
    emerging_words = join_words.takeOrdered(10, lambda wfd: -wfd[1])
    # 10 words that get a decrease in frequency in the sequel
    disappearing_words = join_words.takeOrdered(10, lambda wfd: -wfd[1])
    
    # Print results
    for word, freq_diff in emerging_words:
        print( "%.2f" % (freq_diff*10000), word)
    for word, freq_diff in disappearing_words[::-1]:
        print ("%.2f" % (freq_diff*10000), word)
    

In [6]:
runOpt()

92.52 ulysses
53.63 house
48.33 telemachus
43.06 suitors
36.68 tell
33.47 ship
33.35 one
31.94 home
26.73 said
25.97 got
25.97 got
26.73 said
31.94 home
33.35 one
33.47 ship
36.68 tell
43.06 suitors
48.33 telemachus
53.63 house
92.52 ulysses


## Optimisation Variable globale


return word not in english_stop_words 

long et couteux
La complexité de cette recherche d'éléments est (N)

Passer par une table de hachage
english_stop_words = set(stopwords.words("english"))


Appel par partition si possible: mapPartitions à la palce de filter ou map et en faire une variable global



In [7]:

def filter_stop_words(partition):
    from nltk.corpus import stopwords
    english_stop_words = set(stopwords.words("english"))
    for word in partition:
        if word not in english_stop_words:
            yield word

def load_textOpt(text_path):
    # Split text in words
    # Remove empty word artefacts
    # Remove stop words ('I', 'you', 'a', 'the', ...)
    vocabulary = sc.textFile(text_path)\
        .flatMap(lambda lines: lines.lower().split())\
        .flatMap(lambda word: word.split("."))\
        .flatMap(lambda word: word.split(","))\
        .flatMap(lambda word: word.split("!"))\
        .flatMap(lambda word: word.split("?"))\
        .flatMap(lambda word: word.split("'"))\
        .flatMap(lambda word: word.split("\""))\
        .filter(lambda word: word is not None and len(word) > 0)\
        .mapPartitions(filter_stop_words)\
        .persist()
    # Count the total number of words in the text
    word_count = vocabulary.count()

    # Compute the frequency of each word: frequency = #appearances/#word_count
    word_freq = vocabulary.map(lambda word: (word, 1))\
        .reduceByKey(lambda count1, count2: count1 + count2)\
        .map(lambda wc: (wc[0], wc[1]/float(word_count)))\

    return word_freq


def runOpt():
    iliad = load_textOpt('..\data/iliad.mb.txt')
    odyssey = load_textOpt('..\data/odyssey.mb.txt')
    
    # Join the two datasets and compute the difference in frequency
    # Note that we need to write (freq or 0) because some words do not appear
    # in one of the two books. Thus, some frequencies are equal to None after
    # the full outer join.
    join_words = iliad.fullOuterJoin(odyssey)\
        .map(lambda wf: (wf[0], (wf[1][1] or 0) - (wf[1][0] or 0)))

    
    # 10 words that get a boost in frequency in the sequel
    emerging_words = join_words.takeOrdered(10, lambda wfd: -wfd[1])
    # 10 words that get a decrease in frequency in the sequel
    disappearing_words = join_words.takeOrdered(10, lambda wfd: -wfd[1])
    
    # Print results
    for word, freq_diff in emerging_words:
        print( "%.2f" % (freq_diff*10000), word)
    for word, freq_diff in disappearing_words[::-1]:
        print ("%.2f" % (freq_diff*10000), word)
    

In [8]:
runOpt()

92.52 ulysses
53.63 house
48.33 telemachus
43.06 suitors
36.68 tell
33.47 ship
33.35 one
31.94 home
26.73 said
25.97 got
25.97 got
26.73 said
31.94 home
33.35 one
33.47 ship
36.68 tell
43.06 suitors
48.33 telemachus
53.63 house
92.52 ulysses


## Optimisation Partition


choix du nombre de partition

vocabulary = sc.textFile(os.path.join(current_directory, text_name), minPartitions=4)

In [9]:

def filter_stop_words(partition):
    from nltk.corpus import stopwords
    english_stop_words = set(stopwords.words("english"))
    for word in partition:
        if word not in english_stop_words:
            yield word

def load_textOpt(text_path):
    # Split text in words
    # Remove empty word artefacts
    # Remove stop words ('I', 'you', 'a', 'the', ...)
    vocabulary = sc.textFile(text_path, minPartitions=4)\
        .flatMap(lambda lines: lines.lower().split())\
        .flatMap(lambda word: word.split("."))\
        .flatMap(lambda word: word.split(","))\
        .flatMap(lambda word: word.split("!"))\
        .flatMap(lambda word: word.split("?"))\
        .flatMap(lambda word: word.split("'"))\
        .flatMap(lambda word: word.split("\""))\
        .filter(lambda word: word is not None and len(word) > 0)\
        .mapPartitions(filter_stop_words)\
        .persist()
    # Count the total number of words in the text
    word_count = vocabulary.count()

    # Compute the frequency of each word: frequency = #appearances/#word_count
    word_freq = vocabulary.map(lambda word: (word, 1))\
        .reduceByKey(lambda count1, count2: count1 + count2)\
        .map(lambda wc: (wc[0], wc[1]/float(word_count)))\

    return word_freq


def runOpt():
    iliad = load_textOpt('..\data/iliad.mb.txt')
    odyssey = load_textOpt('..\data/odyssey.mb.txt')
    
    # Join the two datasets and compute the difference in frequency
    # Note that we need to write (freq or 0) because some words do not appear
    # in one of the two books. Thus, some frequencies are equal to None after
    # the full outer join.
    join_words = iliad.fullOuterJoin(odyssey)\
        .map(lambda wf: (wf[0], (wf[1][1] or 0) - (wf[1][0] or 0)))

    
    # 10 words that get a boost in frequency in the sequel
    emerging_words = join_words.takeOrdered(10, lambda wfd: -wfd[1])
    # 10 words that get a decrease in frequency in the sequel
    disappearing_words = join_words.takeOrdered(10, lambda wfd: -wfd[1])
    
    # Print results
    for word, freq_diff in emerging_words:
        print( "%.2f" % (freq_diff*10000), word)
    for word, freq_diff in disappearing_words[::-1]:
        print ("%.2f" % (freq_diff*10000), word)
    

In [10]:
runOpt()

92.52 ulysses
53.63 house
48.33 telemachus
43.06 suitors
36.68 tell
33.47 ship
33.35 one
31.94 home
26.73 said
25.97 got
25.97 got
26.73 said
31.94 home
33.35 one
33.47 ship
36.68 tell
43.06 suitors
48.33 telemachus
53.63 house
92.52 ulysses


In [12]:
sc.stop()